In [1]:
####对原子进行z方向投影，分析原子z方向密度判断层状结构
import numpy
import os
from scipy.signal import argrelextrema
import scipy.signal as sg
from ase.io import read
from ase.io import write
import matplotlib.pyplot as plt

In [2]:
fenbu = numpy.array([0,0.0011,0.0044,0.01,0.0178,0.0278,0.04,0.0544,0.0711,0.09,0.1111,0.1344,0.16,0.1878,0.2178,0.25,0.2844,0.3211,0.36,0.4011,0.4444,0.49,0.5378,0.5878,0.64,0.6944,0.7511,0.81,0.8711,0.9344,1,0.9344,0.8711,0.81,0.7511,0.6944,0.64,0.5878,0.5378,0.49,0.4444,0.4011,0.36,0.3211,0.2844,0.25,0.2178,0.1878,0.16,0.1344,0.1111,0.09,0.0711,0.0544,0.04,0.0278,0.0178,0.01,0.0044,0.0011,0])
####关于z方向2次函数分布

In [3]:
def no_sys(filename):
    sur_atom=read(filename,format='vasp') ####建表面模型时用的数据
    num_atom = numpy.shape(sur_atom.positions)
    z_max = numpy.max(sur_atom.positions[:,2]) ####取第3列最大值  
    z_min = numpy.min(sur_atom.positions[:,2])
    z_max = round(z_max+3,1) ###保留小数点后一位
    z_min = round(z_min-3,1)
    z_d= round((z_max-z_min)*10)
    print(z_max,z_min,z_d)
    labelx = 0 ####给下面的数组坐标计数
    z_p = numpy.zeros([z_d+1,2])
    for i in numpy.arange(z_min,z_max,0.1).round(1): ####普通的range不行 numpy的arange也不行因为是小数步长,不准确，目前只能只能让初始数组长度+1临时解决
        z_p[labelx,0] = round(i,1)       
        labelx = labelx+1
    #####给数组赋值z坐标
    
    for i in range(num_atom[0]):
        zz = round(sur_atom.positions[i,2],1)
        lb = numpy.where(z_p[:,0] == zz )
        for n in range(61):
            z_p[lb[0]+n-31,1] = z_p[lb[0]+n-31,1]+fenbu[n] 
    return z_p

def extract_density_features(filename):
    """
    从vasp文件中提取4个密度分布特征：
    1. densities[max_idx] - 密度分布的最大值
    2. densities[chosen_idx] - 极小值中的最小值
    3. densities[max_idx]/densities[chosen_idx] - 比值
    4. densities[max_idx]-densities[chosen_idx] - 差值
    
    返回: numpy数组包含4个特征，如果处理失败返回None
    """
    from scipy.signal import argrelextrema
    
    try:
        # 1. 计算密度分布
        z_p_current = no_sys(filename)
        
        # 2. 读取原始结构获取原子z坐标范围
        sur_atom = read(filename, format='vasp')
        z_positions = sur_atom.positions[:, 2]
        z_atom_min = z_positions.min()
        z_atom_max = z_positions.max()
        
        # 3. 分析密度分布（排除密度为0的点）
        all_z_coords = z_p_current[:, 0]
        all_densities = z_p_current[:, 1]
        
        mask_nonzero = all_densities > 0
        z_coords = all_z_coords[mask_nonzero]
        densities = all_densities[mask_nonzero]
        
        # 4. 找到密度最大值位置
        max_idx = numpy.argmax(densities)
        density_max = densities[max_idx]
        z_max = z_coords[max_idx]
        
        # 5. 在原子实际存在区域内寻找局部极小值
        search_margin = 0.5
        valid_search_mask = (z_coords >= z_atom_min - search_margin) & (z_coords <= z_atom_max + search_margin)
        
        z_coords_search = z_coords[valid_search_mask]
        densities_search = densities[valid_search_mask]
        search_indices = numpy.where(valid_search_mask)[0]
        
        # 6. 寻找局部极小值
        local_min_indices_search = argrelextrema(densities_search, numpy.less)[0]
        
        if len(local_min_indices_search) > 0:
            # 映射回原始数组的索引
            local_min_indices = search_indices[local_min_indices_search]
            z_local_mins = z_coords[local_min_indices]
            density_local_mins = densities[local_min_indices]
            
            # 找到密度最小的局部极小值
            min_density_idx = numpy.argmin(density_local_mins)
            min_density_value = density_local_mins[min_density_idx]
            
            # 找出所有密度等于最小值的局部极小值
            min_density_mask = density_local_mins == min_density_value
            candidate_indices = local_min_indices[min_density_mask]
            candidate_z = z_local_mins[min_density_mask]
            
            if len(candidate_indices) == 1:
                # 只有一个最小密度的局部极小值
                chosen_idx = candidate_indices[0]
            else:
                # 有多个密度相同的最小局部极小值，选择离最大值最近的
                distances = numpy.abs(candidate_z - z_max)
                closest_among_mins = numpy.argmin(distances)
                chosen_idx = candidate_indices[closest_among_mins]
            
            density_min = densities[chosen_idx]
        else:
            # 如果没有找到局部极小值，使用搜索范围内的全局最小值
            min_idx_search = numpy.argmin(densities_search)
            chosen_idx = search_indices[min_idx_search]
            density_min = densities[chosen_idx]
        
        # 7. 计算4个特征
        features = [
            density_max,                                           # 特征1: 最大值
            density_min,                                           # 特征2: 极小值中的最小值
            density_max / density_min if density_min != 0 else 1.0,  # 特征3: 比值
            density_max - density_min                              # 特征4: 差值
        ]
        
        return numpy.array(features, dtype=numpy.float32)
        
    except Exception as e:
        print(f"Error extracting density features from {filename}: {e}")
        return None

In [4]:
def split_vasp_auto(filename, output_dir='data/split', range_width=3.0, show_plot=True, save_extrema=False):
    """
    自动计算密度分布并分离vasp文件
    
    参数:
    filename: 输入vasp文件路径
    output_dir: 输出根目录，会在其下创建max和min两个子文件夹
    range_width: 提取范围宽度（默认3.0埃）
    show_plot: 是否显示可视化图（默认True）
    save_extrema: 是否保存极值信息到文件（默认False）
    
    返回:
    {
        'output_files': [max_file_path, min_file_path],
        'z_max': 最大值位置,
        'z_min': 选中的极小值位置,
        'all_local_mins': 所有局部极小值列表 [(z, density, distance), ...]
        'skipped': 是否跳过该文件（因为某个区域为空）
    }
    """
    from scipy.signal import argrelextrema
    
    # 创建输出目录结构
    max_dir = os.path.join(output_dir, 'max')
    min_dir = os.path.join(output_dir, 'min')
    
    if not os.path.exists(max_dir):
        os.makedirs(max_dir)
    if not os.path.exists(min_dir):
        os.makedirs(min_dir)
    
    # 获取输入文件的基本名称（不含路径和扩展名）
    base_filename = os.path.basename(filename).replace('.vasp', '')
    
    print(f"{'='*60}")
    print(f"处理文件: {filename}")
    print(f"{'='*60}\n")
    
    # 1. 为该文件计算密度分布
    print("步骤1: 计算Z方向密度分布...")
    z_p_current = no_sys(filename)
    print()
    
    # 2. 读取原始结构
    sur_atom = read(filename, format='vasp')
    z_positions = sur_atom.positions[:, 2]
    z_atom_min = z_positions.min()
    z_atom_max = z_positions.max()
    
    print(f"原始文件原子总数: {len(sur_atom)}")
    print(f"原子z坐标范围: {z_atom_min:.3f} ~ {z_atom_max:.3f} Å\n")
    
    # 3. 分析密度分布（排除密度为0的点）
    all_z_coords = z_p_current[:, 0]
    all_densities = z_p_current[:, 1]
    
    mask_nonzero = all_densities > 0
    z_coords = all_z_coords[mask_nonzero]
    densities = all_densities[mask_nonzero]
    
    print(f"密度分布计算范围: {z_coords.min():.1f} ~ {z_coords.max():.1f} Å")
    print(f"(实际原子范围上下各扩展了3Å以确保分布完整)\n")
    
    # 4. 找到密度最大值位置
    max_idx = numpy.argmax(densities)
    z_max = z_coords[max_idx]
    print(f"步骤2: 寻找密度极值点...")
    print(f"  最大密度位置: z={z_max:.1f} Å, 密度={densities[max_idx]:.3f}")
    
    # *** 关键修改：限制极小值搜索范围在实际原子区域内 ***
    # 5. 过滤出原子实际存在区域内的点（避免边界效应）
    # 使用原子范围，再向外稍微扩展0.5Å作为搜索范围
    search_margin = 0.5
    valid_search_mask = (z_coords >= z_atom_min - search_margin) & (z_coords <= z_atom_max + search_margin)
    
    z_coords_search = z_coords[valid_search_mask]
    densities_search = densities[valid_search_mask]
    search_indices = numpy.where(valid_search_mask)[0]  # 记录在原数组中的索引
    
    print(f"\n  极小值搜索范围限制:")
    print(f"    原子范围: {z_atom_min:.2f} ~ {z_atom_max:.2f} Å")
    print(f"    搜索范围: {z_atom_min - search_margin:.2f} ~ {z_atom_max + search_margin:.2f} Å")
    print(f"    (避免边界处的低密度点被误识别为极小值)")
    
    # 6. 在限定范围内寻找局部极小值
    local_min_indices_search = argrelextrema(densities_search, numpy.less)[0]
    
    all_local_mins = []  # 存储所有局部极小值信息
    
    if len(local_min_indices_search) > 0:
        # 映射回原始数组的索引
        local_min_indices = search_indices[local_min_indices_search]
        z_local_mins = z_coords[local_min_indices]
        density_local_mins = densities[local_min_indices]
        
        print(f"\n  {'='*56}")
        print(f"  在搜索范围内找到 {len(local_min_indices)} 个局部极小值:")
        print(f"  {'='*56}")
        print(f"  {'序号':<6} {'Z坐标(Å)':<12} {'密度':<12} {'距最大值(Å)':<15}")
        print(f"  {'-'*56}")
        
        for i, (idx, z_pos, dens) in enumerate(zip(local_min_indices, z_local_mins, density_local_mins), 1):
            dist = abs(z_pos - z_max)
            all_local_mins.append((z_pos, dens, dist))
            print(f"  {i:<6} {z_pos:<12.2f} {dens:<12.4f} {dist:<15.2f}")
        
        print(f"  {'='*56}\n")
        
        # 找到密度最小的局部极小值
        min_density_idx = numpy.argmin(density_local_mins)
        min_density_value = density_local_mins[min_density_idx]
        
        # 找出所有密度等于最小值的局部极小值
        min_density_mask = density_local_mins == min_density_value
        candidate_indices = local_min_indices[min_density_mask]
        candidate_z = z_local_mins[min_density_mask]
        
        print(f"  选择逻辑:")
        if len(candidate_indices) == 1:
            # 只有一个最小密度的局部极小值
            chosen_idx = candidate_indices[0]
            z_min = candidate_z[0]
            print(f"    ✓ 唯一的最小密度局部极小值:")
            print(f"      z={z_min:.2f} Å, 密度={min_density_value:.4f}")
        else:
            # 有多个密度相同的最小局部极小值，选择离最大值最近的
            distances = numpy.abs(candidate_z - z_max)
            closest_among_mins = numpy.argmin(distances)
            chosen_idx = candidate_indices[closest_among_mins]
            z_min = candidate_z[closest_among_mins]
            
            print(f"    找到 {len(candidate_indices)} 个密度相同的最小局部极小值（密度={min_density_value:.4f}）:")
            for i, (idx, z_pos) in enumerate(zip(candidate_indices, candidate_z)):
                dist = abs(z_pos - z_max)
                marker = " ← 选中（最近）" if idx == chosen_idx else ""
                print(f"      - z={z_pos:.2f} Å, 距离最大值={dist:.2f} Å{marker}")
            print(f"    ✓ 选择离最大值最近的:")
            print(f"      z={z_min:.2f} Å, 密度={densities[chosen_idx]:.4f}, 距离={abs(z_min-z_max):.2f} Å")
        
        print()
    else:
        # 如果在搜索范围内没有找到局部极小值，使用该范围内的全局最小值
        min_idx_search = numpy.argmin(densities_search)
        min_idx = search_indices[min_idx_search]
        z_min = z_coords[min_idx]
        chosen_idx = min_idx
        all_local_mins.append((z_min, densities[min_idx], abs(z_min - z_max)))
        print(f"\n  在搜索范围内未找到局部极小值，使用该范围内的全局最小值:")
        print(f"  z={z_min:.2f} Å, 密度={densities[min_idx]:.4f}\n")
    
    # 7. 动态调整max区域的range_width
    # 获取z_max边界处的密度
    def get_density_at_z(z_target):
        """获取指定z坐标处的密度值（使用最近的点）"""
        idx = numpy.argmin(numpy.abs(z_coords - z_target))
        return densities[idx]
    
    def find_boundary_at_density(target_density, side):
        """
        找到密度小于等于target_density的边界点
        side: 'left' (z < z_max), 'right' (z > z_max)
        返回: 最接近z_max的满足条件的z坐标
        """
        if side == 'left':
            # 左侧：找所有 z < z_max 且 density <= target_density 的点
            mask = (z_coords < z_max) & (densities <= target_density)
            if not numpy.any(mask):
                return None
            valid_z = z_coords[mask]
            # 返回z值最大的（最接近z_max）
            return numpy.max(valid_z)
        elif side == 'right':
            # 右侧：找所有 z > z_max 且 density <= target_density 的点
            mask = (z_coords > z_max) & (densities <= target_density)
            if not numpy.any(mask):
                return None
            valid_z = z_coords[mask]
            # 返回z值最小的（最接近z_max）
            return numpy.min(valid_z)
        return None
    
    density_at_z_min = densities[chosen_idx]
    density_left_boundary = get_density_at_z(z_max - range_width)
    density_right_boundary = get_density_at_z(z_max + range_width)
    
    # 检查max和min区域是否重叠
    distance_between_peaks = abs(z_max - z_min)
    regions_overlap = distance_between_peaks < 2 * range_width
    
    max_range_adjusted = False
    actual_range_width = range_width
    z_max_left = z_max - range_width
    z_max_right = z_max + range_width
    
    print(f"步骤3: 检查max区域的range_width...")
    print(f"  默认range_width: ±{range_width} Å")
    print(f"  z_max与z_min距离: {distance_between_peaks:.2f} Å")
    print(f"  区域重叠检测: {'重叠' if regions_overlap else '不重叠'}")
    print(f"  z_min处密度: {density_at_z_min:.4f}")
    print(f"  z_max左边界({z_max - range_width:.1f})处密度: {density_left_boundary:.4f}")
    print(f"  z_max右边界({z_max + range_width:.1f})处密度: {density_right_boundary:.4f}")
    
    # 判断是否需要调整range_width
    needs_adjustment = (
        density_left_boundary < density_at_z_min or 
        density_right_boundary < density_at_z_min or
        regions_overlap
    )
    
    if needs_adjustment:
        if regions_overlap:
            print(f"  ⚠ 检测到max区域与min区域重叠，需要调整max区域范围")
        else:
            print(f"  ⚠ 检测到边界密度低于z_min处密度，需要调整max区域范围")
        print(f"  使用新方法：找到密度<=z_min*105%且最接近z_max的边界点")
        max_range_adjusted = True
        
        # 使用选中的z_min来计算目标密度
        target_density = density_at_z_min * 1.05
        
        print(f"    选中的z_min密度: {density_at_z_min:.4f}")
        print(f"    目标密度阈值(105%): {target_density:.4f}")
        
        # 查找左侧边界
        z_max_left_candidate = find_boundary_at_density(target_density, side='left')
        
        if z_max_left_candidate is not None:
            z_max_left = z_max_left_candidate
            print(f"    ✓ 左边界: z={z_max_left:.1f} (距离z_max={abs(z_max - z_max_left):.2f}Å)")
        else:
            print(f"    左侧：未找到密度<={target_density:.4f}的点，保持原范围")
            z_max_left = z_max - range_width
        
        # 查找右侧边界
        z_max_right_candidate = find_boundary_at_density(target_density, side='right')
        
        if z_max_right_candidate is not None:
            z_max_right = z_max_right_candidate
            print(f"    ✓ 右边界: z={z_max_right:.1f} (距离z_max={abs(z_max_right - z_max):.2f}Å)")
        else:
            print(f"    右侧：未找到密度<={target_density:.4f}的点，保持原范围")
            z_max_right = z_max + range_width
        
        actual_range_width = max(abs(z_max_right - z_max), abs(z_max - z_max_left))
        print(f"  ✓ max区域范围已调整: z={z_max_left:.1f} ~ {z_max_right:.1f} Å")
    else:
        print(f"  ✓ 边界密度正常且无重叠，使用默认range_width")
    
    print()
    
    # 8. 检查两个区域的原子数（在保存前先检查）
    print(f"步骤4: 检查提取区域...")
    
    # 区域1: 最大值附近
    mask1 = (z_positions >= z_max_left) & (z_positions <= z_max_right)
    atoms_in_region1 = numpy.sum(mask1)
    
    # 区域2: 选中的极小值附近
    mask2 = numpy.abs(z_positions - z_min) <= range_width
    atoms_in_region2 = numpy.sum(mask2)
    
    print(f"  区域1（最大密度附近）:")
    print(f"    中心位置: z={z_max:.1f} Å")
    if max_range_adjusted:
        print(f"    提取范围: z={z_max_left:.1f} ~ {z_max_right:.1f} Å (已调整)")
    else:
        print(f"    提取范围: z={z_max_left:.1f} ~ {z_max_right:.1f} Å")
    print(f"    找到原子数: {atoms_in_region1}")
    
    print(f"  区域2（选中的极小值附近）:")
    print(f"    中心位置: z={z_min:.1f} Å")
    print(f"    提取范围: z={z_min-range_width:.1f} ~ {z_min+range_width:.1f} Å")
    print(f"    找到原子数: {atoms_in_region2}")
    
    # *** 关键修改：如果任何一个区域为空，跳过整个文件 ***
    if atoms_in_region1 == 0 or atoms_in_region2 == 0:
        print(f"\n  ✗ 检测到至少一个区域为空，跳过该文件的处理")
        print(f"{'='*60}")
        print(f"文件已跳过（区域为空）")
        print(f"{'='*60}\n")
        
        # 返回跳过状态
        result = {
            'output_files': [None, None],
            'z_max': z_max,
            'z_min': z_min,
            'density_max': densities[max_idx],
            'density_min': densities[chosen_idx],
            'all_local_mins': all_local_mins,
            'num_local_mins': len(all_local_mins),
            'atoms_region1': atoms_in_region1,
            'atoms_region2': atoms_in_region2,
            'max_range_adjusted': max_range_adjusted,
            'z_max_left': z_max_left,
            'z_max_right': z_max_right,
            'regions_overlap': regions_overlap,
            'skipped': True,
            'skip_reason': 'empty_region'
        }
        return result
    
    print(f"\n  ✓ 两个区域都有原子，继续处理\n")
    
    # 9. 保存极值信息到文件（可选）
    if save_extrema and len(all_local_mins) > 0:
        extrema_file = os.path.join(output_dir, f"{base_filename}_extrema.txt")
        with open(extrema_file, 'w', encoding='utf-8') as f:
            f.write(f"VASP文件极值分析报告\n")
            f.write(f"{'='*60}\n")
            f.write(f"输入文件: {filename}\n")
            f.write(f"原始文件原子总数: {len(sur_atom)}\n")
            f.write(f"原子z坐标范围: {z_positions.min():.3f} ~ {z_positions.max():.3f} Å\n")
            f.write(f"密度分布计算范围: {z_coords.min():.1f} ~ {z_coords.max():.1f} Å\n")
            f.write(f"极小值搜索范围: {z_atom_min - search_margin:.2f} ~ {z_atom_max + search_margin:.2f} Å\n")
            f.write(f"\n")
            f.write(f"最大密度位置:\n")
            f.write(f"  z = {z_max:.2f} Å\n")
            f.write(f"  密度 = {densities[max_idx]:.4f}\n")
            f.write(f"\n")
            f.write(f"所有局部极小值 (共 {len(all_local_mins)} 个):\n")
            f.write(f"{'序号':<6} {'Z坐标(Å)':<12} {'密度':<12} {'距最大值(Å)':<15}\n")
            f.write(f"{'-'*50}\n")
            for i, (z_pos, dens, dist) in enumerate(all_local_mins, 1):
                marker = " *选中" if abs(z_pos - z_min) < 0.01 else ""
                f.write(f"{i:<6} {z_pos:<12.2f} {dens:<12.4f} {dist:<15.2f}{marker}\n")
            f.write(f"\n")
            f.write(f"选中的极小值:\n")
            f.write(f"  z = {z_min:.2f} Å\n")
            f.write(f"  密度 = {densities[chosen_idx]:.4f}\n")
            f.write(f"  距离最大值 = {abs(z_min - z_max):.2f} Å\n")
            f.write(f"\n")
            f.write(f"提取范围宽度: ±{range_width} Å\n")
            if max_range_adjusted:
                f.write(f"max区域已调整: z={z_max_left:.1f} ~ {z_max_right:.1f} Å\n")
            f.write(f"{'='*60}\n")
        print(f"  ✓ 极值信息已保存到: {extrema_file}\n")
    
    # 10. 提取并保存两个区域的原子
    print(f"步骤5: 保存提取的原子...")
    output_files = []
    
    # 区域1: 最大值附近 -> 保存到max文件夹
    z_in_region1 = z_positions[mask1]
    print(f"  区域1（最大密度附近）:")
    print(f"    实际原子z范围: {z_in_region1.min():.3f} ~ {z_in_region1.max():.3f} Å")
    atoms1 = sur_atom[mask1]
    # 清除速度信息
    if atoms1.has('momenta'):
        atoms1.set_momenta(None)
    output1 = os.path.join(max_dir, f"{base_filename}.vasp")
    write(output1, atoms1, format='vasp', vasp5=True, direct=False)
    print(f"    ✓ 已保存到: {output1}\n")
    output_files.append(output1)
    
    # 区域2: 选中的极小值附近 -> 保存到min文件夹
    z_in_region2 = z_positions[mask2]
    print(f"  区域2（选中的极小值附近）:")
    print(f"    实际原子z范围: {z_in_region2.min():.3f} ~ {z_in_region2.max():.3f} Å")
    atoms2 = sur_atom[mask2]
    # 清除速度信息
    if atoms2.has('momenta'):
        atoms2.set_momenta(None)
    output2 = os.path.join(min_dir, f"{base_filename}.vasp")
    write(output2, atoms2, format='vasp', vasp5=True, direct=False)
    print(f"    ✓ 已保存到: {output2}\n")
    output_files.append(output2)
    
    # 11. 可视化
    if show_plot:
        plt.figure(figsize=(14, 7))
        plt.plot(z_coords, densities, linewidth=2, label='Atomic Density', color='blue')
        
        # 标记实际原子位置
        for z_pos in z_positions:
            plt.axvline(z_pos, color='gray', alpha=0.15, linewidth=0.5)
        
        # 标记搜索范围边界
        plt.axvline(z_atom_min - search_margin, color='purple', linestyle=':', linewidth=1.5, 
                   alpha=0.5, label=f'Search range boundary')
        plt.axvline(z_atom_max + search_margin, color='purple', linestyle=':', linewidth=1.5, alpha=0.5)
        
        # 标记所有局部极小值
        if len(all_local_mins) > 0:
            for z_pos, dens, dist in all_local_mins:
                if abs(z_pos - z_min) < 0.01:
                    continue  # 选中的极小值单独标记
                plt.scatter([z_pos], [dens], color='orange', s=80, 
                           zorder=5, alpha=0.6, marker='o',
                           edgecolors='darkorange', linewidth=1,
                           label='Other local minima')
        
        # 标记提取区域
        plt.axvline(z_max, color='red', linestyle='--', linewidth=2.5, alpha=0.8)
        plt.scatter([z_max], [densities[max_idx]], color='red', s=250, 
                   marker='*', zorder=7, edgecolors='darkred', linewidth=2,
                   label=f'Max: z={z_max:.1f}Å, ρ={densities[max_idx]:.3f}')
        
        # 根据是否调整显示不同的区域
        if max_range_adjusted:
            plt.axvspan(z_max_left, z_max_right, alpha=0.15, color='red', 
                        label=f'Region 1: {atoms_in_region1} atoms (adjusted)')
        else:
            plt.axvspan(z_max_left, z_max_right, alpha=0.15, color='red', 
                        label=f'Region 1: {atoms_in_region1} atoms')
        
        plt.axvline(z_min, color='green', linestyle='--', linewidth=2.5, alpha=0.8)
        plt.scatter([z_min], [densities[chosen_idx]], color='green', s=250, 
                   marker='*', zorder=7, edgecolors='darkgreen', linewidth=2,
                   label=f'Selected Min: z={z_min:.1f}Å, ρ={densities[chosen_idx]:.3f}')
        plt.axvspan(z_min-range_width, z_min+range_width, alpha=0.15, color='green', 
                    label=f'Region 2: {atoms_in_region2} atoms')
        
        plt.xlabel('Z Coordinate (Å)', fontsize=13, fontweight='bold')
        plt.ylabel('Atomic Density', fontsize=13, fontweight='bold')
        plt.title(f'VASP File Splitting: {filename.split("/")[-1]}', fontsize=15, fontweight='bold')
        plt.legend(fontsize=9, loc='best', framealpha=0.9)
        plt.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.show()
    
    print(f"{'='*60}")
    print(f"处理完成！")
    print(f"{'='*60}\n")
    
    # 返回详细结果
    result = {
        'output_files': output_files,
        'z_max': z_max,
        'z_min': z_min,
        'density_max': densities[max_idx],
        'density_min': densities[chosen_idx],
        'all_local_mins': all_local_mins,
        'num_local_mins': len(all_local_mins),
        'atoms_region1': atoms_in_region1,
        'atoms_region2': atoms_in_region2,
        'max_range_adjusted': max_range_adjusted,
        'z_max_left': z_max_left,
        'z_max_right': z_max_right,
        'regions_overlap': regions_overlap,
        'skipped': False
    }
    
    return result

In [ ]:
# 批量处理整个文件夹中的所有vasp文件
import glob

# 配置参数
input_folder = 'data/sur_last'          # 输入文件夹路径
output_folder = 'data/split/sur_last'       # 输出根目录（会创建max和min子文件夹）
range_width = 3.0                          # 提取范围宽度（埃）
show_plot = False                          # 是否显示每个文件的图形（批量处理建议False）

# 获取文件夹中所有的vasp文件
vasp_files = glob.glob(os.path.join(input_folder, '*.vasp'))

if len(vasp_files) == 0:
    print(f"警告：在 {input_folder} 中没有找到vasp文件！")
else:
    print(f"{'#'*70}")
    print(f"# 批量处理开始")
    print(f"{'#'*70}")
    print(f"输入目录: {input_folder}")
    print(f"输出目录: {output_folder}")
    print(f"  - 最大值区域: {os.path.join(output_folder, 'max')}")
    print(f"  - 极小值区域: {os.path.join(output_folder, 'min')}")
    print(f"找到 {len(vasp_files)} 个vasp文件")
    print(f"{'#'*70}\n")
    
    success_count = 0
    fail_count = 0
    skip_count = 0  # 新增：跳过的文件计数
    adjusted_count = 0  # 统计使用了动态调整的文件数
    failed_files = []
    skipped_files = []  # 新增：跳过的文件列表
    
    for i, filepath in enumerate(vasp_files, 1):
        try:
            print(f"[{i}/{len(vasp_files)}] 处理: {os.path.basename(filepath)}")
            
            # 调用处理函数
            result = split_vasp_auto(
                filename=filepath,
                output_dir=output_folder,
                range_width=range_width,
                show_plot=show_plot
            )
            
            # 检查是否跳过了该文件
            if result.get('skipped', False):
                skip_count += 1
                skip_reason = "区域为空" if result.get('skip_reason') == 'empty_region' else "未知原因"
                skipped_files.append((os.path.basename(filepath), skip_reason))
                print(f"⊘ 文件已跳过 ({skip_reason})\n")
            # 统计成功的文件
            elif any(f is not None for f in result['output_files']):
                success_count += 1
                if result['max_range_adjusted']:
                    adjusted_count += 1
                    print(f"✓ 成功处理 (max区域已动态调整)\n")
                else:
                    print(f"✓ 成功处理\n")
            else:
                fail_count += 1
                failed_files.append((os.path.basename(filepath), "未生成输出文件"))
                print(f"✗ 警告：未生成输出文件\n")
                
        except Exception as e:
            print(f"✗ 错误：{str(e)}\n")
            fail_count += 1
            failed_files.append((os.path.basename(filepath), str(e)))
    
    # 输出总结
    print(f"\n{'#'*70}")
    print(f"# 批量处理完成")
    print(f"{'#'*70}")
    print(f"总文件数: {len(vasp_files)}")
    print(f"成功: {success_count}")
    print(f"跳过: {skip_count} (因区域为空)")
    print(f"失败: {fail_count}")
    print(f"使用动态调整: {adjusted_count}")
    
    if skipped_files:
        print(f"\n跳过的文件列表 (共{skip_count}个):")
        for fname, reason in skipped_files:
            print(f"  - {fname}: {reason}")
    
    if failed_files:
        print(f"\n失败的文件列表 (共{fail_count}个):")
        for fname, error in failed_files:
            print(f"  - {fname}: {error}")
    
    print(f"\n输出目录结构:")
    print(f"  {output_folder}/")
    print(f"    ├── max/  ({success_count} 个文件)")
    print(f"    └── min/  ({success_count} 个文件)")
    print(f"{'#'*70}")

In [ ]:
# 检测转换为空的文件
import glob

# 配置参数
input_folder = 'data/sur_last'          # 输入文件夹路径
output_folder = 'data/split/sur_last'       # 输出根目录
range_width = 3.0                          # 提取范围宽度（埃）

# 获取文件夹中所有的vasp文件
vasp_files = glob.glob(os.path.join(input_folder, '*.vasp'))

if len(vasp_files) == 0:
    print(f"警告：在 {input_folder} 中没有找到vasp文件！")
else:
    print(f"{'='*80}")
    print(f"检测转换为空的文件")
    print(f"{'='*80}")
    print(f"输入目录: {input_folder}")
    print(f"总文件数: {len(vasp_files)}")
    print(f"{'='*80}\n")
    
    empty_files = []  # 存储转换为空的文件信息
    
    for i, filepath in enumerate(vasp_files, 1):
        try:
            # 静默处理（不显示详细输出和图表）
            import sys
            from io import StringIO
            
            # 捕获标准输出
            old_stdout = sys.stdout
            sys.stdout = StringIO()
            
            result = split_vasp_auto(
                filename=filepath,
                output_dir=output_folder,
                range_width=range_width,
                show_plot=False
            )
            
            # 恢复标准输出
            sys.stdout = old_stdout
            
            # 检查是否有区域为空
            filename = os.path.basename(filepath)
            region1_empty = result['atoms_region1'] == 0
            region2_empty = result['atoms_region2'] == 0
            
            if region1_empty or region2_empty:
                empty_info = {
                    'filename': filename,
                    'filepath': filepath,
                    'region1_atoms': result['atoms_region1'],
                    'region2_atoms': result['atoms_region2'],
                    'z_max': result['z_max'],
                    'z_min': result['z_min'],
                    'distance': abs(result['z_max'] - result['z_min']),
                    'overlap': result['regions_overlap'],
                    'adjusted': result['max_range_adjusted']
                }
                empty_files.append(empty_info)
                
                # 实时显示发现的空文件
                print(f"[{i}/{len(vasp_files)}] ✗ 发现空区域: {filename}")
                if region1_empty:
                    print(f"             区域1（max）为空")
                if region2_empty:
                    print(f"             区域2（min）为空")
            else:
                # 静默跳过正常文件
                pass
                
        except Exception as e:
            filename = os.path.basename(filepath)
            print(f"[{i}/{len(vasp_files)}] ✗ 处理错误: {filename} - {str(e)}")
    
    # 输出详细统计
    print(f"\n{'='*80}")
    print(f"检测完成！")
    print(f"{'='*80}")
    print(f"转换为空的文件数量: {len(empty_files)}/{len(vasp_files)}")
    print(f"{'='*80}\n")
    
    if empty_files:
        print(f"{'='*80}")
        print(f"转换为空的文件详细列表")
        print(f"{'='*80}")
        print(f"{'序号':<6} {'文件名':<40} {'区域1':<8} {'区域2':<8} {'距离(Å)':<10} {'重叠':<6}")
        print(f"{'-'*80}")
        
        for i, info in enumerate(empty_files, 1):
            region1_str = f"{info['region1_atoms']}" if info['region1_atoms'] > 0 else "空"
            region2_str = f"{info['region2_atoms']}" if info['region2_atoms'] > 0 else "空"
            overlap_str = "是" if info['overlap'] else "否"
            
            print(f"{i:<6} {info['filename']:<40} {region1_str:<8} {region2_str:<8} "
                  f"{info['distance']:<10.2f} {overlap_str:<6}")
        
        print(f"{'='*80}\n")
        
        # 分类统计
        region1_empty_count = sum(1 for info in empty_files if info['region1_atoms'] == 0)
        region2_empty_count = sum(1 for info in empty_files if info['region2_atoms'] == 0)
        both_empty_count = sum(1 for info in empty_files if info['region1_atoms'] == 0 and info['region2_atoms'] == 0)
        
        print(f"分类统计:")
        print(f"  - 仅区域1（max）为空: {region1_empty_count - both_empty_count}")
        print(f"  - 仅区域2（min）为空: {region2_empty_count - both_empty_count}")
        print(f"  - 两个区域都为空: {both_empty_count}")
        
        # 导出为文本文件（可选）
        output_file = os.path.join(output_folder, 'empty_files_report.txt')
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"转换为空的文件报告\n")
            f.write(f"{'='*80}\n")
            f.write(f"检测时间: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"输入目录: {input_folder}\n")
            f.write(f"总文件数: {len(vasp_files)}\n")
            f.write(f"转换为空的文件数: {len(empty_files)}\n")
            f.write(f"{'='*80}\n\n")
            
            f.write(f"{'序号':<6} {'文件名':<40} {'区域1':<8} {'区域2':<8} {'z_max':<10} {'z_min':<10} {'距离(Å)':<10} {'重叠':<6} {'调整':<6}\n")
            f.write(f"{'-'*120}\n")
            
            for i, info in enumerate(empty_files, 1):
                region1_str = f"{info['region1_atoms']}" if info['region1_atoms'] > 0 else "空"
                region2_str = f"{info['region2_atoms']}" if info['region2_atoms'] > 0 else "空"
                overlap_str = "是" if info['overlap'] else "否"
                adjusted_str = "是" if info['adjusted'] else "否"
                
                f.write(f"{i:<6} {info['filename']:<40} {region1_str:<8} {region2_str:<8} "
                        f"{info['z_max']:<10.2f} {info['z_min']:<10.2f} {info['distance']:<10.2f} "
                        f"{overlap_str:<6} {adjusted_str:<6}\n")
            
            f.write(f"\n{'='*80}\n")
            f.write(f"分类统计:\n")
            f.write(f"  - 仅区域1（max）为空: {region1_empty_count - both_empty_count}\n")
            f.write(f"  - 仅区域2（min）为空: {region2_empty_count - both_empty_count}\n")
            f.write(f"  - 两个区域都为空: {both_empty_count}\n")
            f.write(f"{'='*80}\n")
        
        print(f"\n✓ 详细报告已保存到: {output_file}")
        
    else:
        print(f"✓ 没有发现转换为空的文件！所有文件都成功提取。")
